In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q -U trl transformers datasets==4.3.0 accelerate bitsandbytes

In [ ]:
from huggingface_hub import login
import wandb
import os


HF_TOKEN = "Your token"
login(token=HF_TOKEN)

WANDB_KEY = "Your token"
wandb.login(key=WANDB_KEY)

%env WANDB_PROJECT=Llama-3.2-1B-DPO-Alignment

In [8]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 1024 
dtype = None
load_in_4bit = True

SFT_MODEL_NAME = "Phat-Dat/Llama-3.2-1B-Instruct-Chat-sft"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=SFT_MODEL_NAME,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth", 
    random_state=3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-11-11 04:08:29.265604: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762834109.493648      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762834109.580299      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth 2025.11.3 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [10]:
from datasets import load_dataset

DATASET_NAME = "thuanan/Vi-Alpaca-Preference"
dataset = load_dataset(DATASET_NAME, split="train")

SYS_INSTRUCT = "Bạn là một trợ lý AI thân thiện, hãy trả lời bằng tiếng Việt."

def format_dpo_dataset(example):
    """
    Định dạng prompt để khớp với chat template cho DPO.
    DPOTrainer cần 3 cột:
    1. prompt: Cuộc trò chuyện TỚI TRƯỚC lượt trả lời của assistant.
    2. chosen: Câu trả lời ĐƯỢC CHỌN (dạng string).
    3. rejected: Câu trả lời BỊ TỪ CHỐI (dạng string).
    """

    prompt_messages = [
        {"role": "system", "content": SYS_INSTRUCT},
        {"role": "user", "content": str(example['prompt'])}
    ]

    example['prompt'] = tokenizer.apply_chat_template(
        prompt_messages,
        tokenize=False,
        add_generation_prompt=True
    )

    example['chosen'] = str(example['chosen'])
    example['rejected'] = str(example['rejected'])

    return example

dataset = dataset.map(format_dpo_dataset, num_proc=4)

print("\n[PROMPT]:\n" + dataset[0]['prompt'])
print("\n[CHOSEN]:\n" + dataset[0]['chosen'])
print("\n[REJECTED]:\n" + dataset[0]['rejected'])

README.md:   0%|          | 0.00/169 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/76.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/65017 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/65017 [00:00<?, ? examples/s]


[PROMPT]:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 11 Nov 2025

Bạn là một trợ lý AI thân thiện, hãy trả lời bằng tiếng Việt.<|eot_id|><|start_header_id|>user<|end_header_id|>

Xác định và sửa lỗi ngữ pháp.

Tôi đã đi đến cửa hàng.<|eot_id|><|start_header_id|>assistant<|end_header_id|>



[CHOSEN]:
[{'content': 'Xác định và sửa lỗi ngữ pháp.\n\nTôi đã đi đến cửa hàng.', 'role': 'user'}, {'content': 'Không có lỗi ngữ pháp. Câu này đã chính xác.', 'role': 'assistant'}]

[REJECTED]:
[{'content': 'Xác định và sửa lỗi ngữ pháp.\n\nTôi đã đi đến cửa hàng.', 'role': 'user'}, {'content': 'Câu này không có lỗi ngữ pháp.', 'role': 'assistant'}]


In [11]:
from trl import DPOConfig, DPOTrainer

OUTPUT_DIR = "./checkpoint/Llama-3.2-1B-dpo-unsloth"

ppo_actor_lr = 5e-7
ppo_kl_coef = 0.01
ppo_micro_batch = 2
ppo_total_batch = 32
grad_acc_steps = ppo_total_batch // ppo_micro_batch

training_args = DPOConfig(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=ppo_micro_batch,
    gradient_accumulation_steps=grad_acc_steps,
    num_train_epochs=1,                
    learning_rate=ppo_actor_lr,         # 5e-7
    beta=ppo_kl_coef,                   # 0.01

    #  Unsloth /QoL
    optim="adamw_bnb_8bit",           
    logging_steps=20,
    save_strategy="steps",
    save_steps=50,
    fp16=True,                         
    max_length=max_seq_length,          
    max_prompt_length=max_seq_length // 2, 
    gradient_checkpointing=True,       
    dataloader_num_workers=2,

    report_to="wandb",
    run_name="llama-1b-dpo-unsloth-v1",
)

# Trainer 
dpo_trainer = DPOTrainer(
    model=model,
    ref_model=None, 
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

print("Starting DPO training...")
dpo_trainer.train()

print("Training completed!")

dpo_trainer.save_model(OUTPUT_DIR)
print(f"Adapters saved at: {OUTPUT_DIR}")

Extracting prompt in train dataset (num_proc=8):   0%|          | 0/65017 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=8):   0%|          | 0/65017 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=8):   0%|          | 0/65017 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Starting DPO training...


...

wandb: Syncing run llama-1b-dpo-unsloth-v1


wandb: ⭐️ View project at https://wandb.ai/tophatdat-160305-ai-viet-nam/Llama-3.2-1B-DPO-Alignment


wandb: 🚀 View run at https://wandb.ai/tophatdat-160305-ai-viet-nam/Llama-3.2-1B-DPO-Alignment/runs/l8pd9g0e


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss
20,0.693100,0.000047,0.000015,0.354688,0.000033,-338.933594,-466.469330,0.234115,0.193565,0,0,0
40,0.693100,0.000033,0.000002,0.521875,0.000031,-369.816986,-492.195892,0.254837,0.221155,No Log,No Log,No Log
60,0.693100,-0.000104,-0.000117,0.518750,0.000012,-338.725433,-450.274597,0.253750,0.216197,No Log,No Log,No Log
80,0.693100,-0.000182,-0.000341,0.603125,0.000158,-335.277740,-450.679688,0.250204,0.208923,No Log,No Log,No Log
100,0.693000,-0.000364,-0.000632,0.620313,0.000268,-332.776642,-441.803619,0.233604,0.188157,No Log,No Log,No Log
120,0.692900,-0.000653,-0.001173,0.689062,0.000520,-336.923492,-445.391785,0.247101,0.209725,No Log,No Log,No Log
140,0.692700,-0.001082,-0.002010,0.739062,0.000928,-359.346436,-470.009277,0.247120,0.207037,No Log,No Log,No Log
160,0.692400,-0.001666,-0.003172,0.775000,0.001507,-356.240417,-463.703033,0.266369,0.223869,No Log,No Log,No Log
180,0.691900,-0.002377,-0.004797,0.825000,0.002420,-357.048004,-470.502930,0.236660,0.211976,No Log,No Log,No Log
200,0.691400,-0.003210,-0.006695,0.857813,0.003486,-357.223663,-462.655121,0.207860,0.185026,No Log,No Log,No Log


In [ ]:
import torch
del dpo_trainer
torch.cuda.empty_cache()

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=OUTPUT_DIR,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

merged_output_dir = "./checkpoint/Llama-3.2-1B-RLHF-DPO-merged"

model.save_pretrained_merged(
    merged_output_dir,
    tokenizer,
    save_method="merged_16bit", 
)

print(f"Merged model saved at: {merged_output_dir}")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

final_model = AutoModelForCausalLM.from_pretrained(merged_output_dir)
final_tokenizer = AutoTokenizer.from_pretrained(merged_output_dir)

YOUR_USERNAME = "Phat-Dat"
REPO_NAME = "Llama-3.2-1B-RLHF-DPO"

print("Pushing to Hub...")
final_model.push_to_hub(
    f"{YOUR_USERNAME}/{REPO_NAME}",
    commit_message="Add DPO (RLHF) aligned model ckpt",
    token=HF_TOKEN
)

final_tokenizer.push_to_hub(
    f"{YOUR_USERNAME}/{REPO_NAME}",
    commit_message="Add tokenizer",
    token=HF_TOKEN
)

print("Complete!")